## NASA POWER API

### Parameter check

In [26]:
import requests
import pandas as pd
import datetime
import time

In [ ]:
POWER_URL = "https://power.larc.nasa.gov/api/temporal/daily/point"
param_df = pd.read_csv("nasa_daily_parameters.csv")

# 2) ใช้ช่วงเวลาสั้น ๆ พอสำหรับ test (เช่น 1 ปีล่าสุด)
end_date = datetime.datetime.today()
start_date = end_date - datetime.timedelta(days=365)

start_str = start_date.strftime("%Y%m%d")
end_str = end_date.strftime("%Y%m%d")

LAT = 13.736717
LON = 100.523186

print(f"Test period: {start_str} → {end_str}")
print(f"Location: lat={LAT}, lon={LON}")
print(f"Total parameters to test: {len(param_df)}\n")

Test period: 20241119 → 20251119
Location: lat=15.0, lon=100.0
Total parameters to test: 139



In [ ]:
results = []

for i, row in param_df.iterrows():
    param_name = row["parameter"]

    query = {
        "parameters": param_name,
        "community": "AG",
        "longitude": LON,
        "latitude": LAT,
        "start": start_str,
        "end": end_str,
        "format": "JSON",
    }

    print(f"[{i+1}/{len(param_df)}] Testing parameter: {param_name} ...", end=" ")

    try:
        r = requests.get(POWER_URL, params=query, timeout=30)
        status = r.status_code

        if status != 200:
            print(f"✗ HTTP {status}")
            results.append(
                {
                    "parameter": param_name,
                    "status_code": status,
                    "ok": False,
                    "message": f"HTTP {status}",
                }
            )
            # เผื่อ 422 / 400 อยากเก็บข้อความ error
            # results[-1]["error_text"] = r.text[:200]
            continue

        data = r.json()

        # ลองดึงค่าจริงจาก properties.parameter
        param_block = data.get("properties", {}).get("parameter", {})
        values = param_block.get(param_name)

        if values is None or len(values) == 0:
            # API ตอบ 200 แต่ไม่มีค่า
            print("!? 200 แต่ไม่มี data")
            results.append(
                {
                    "parameter": param_name,
                    "status_code": status,
                    "ok": False,
                    "message": "No data returned",
                }
            )
        else:
            print("✓ OK")
            results.append(
                {
                    "parameter": param_name,
                    "status_code": status,
                    "ok": True,
                    "message": "OK",
                }
            )

    except Exception as e:
        print(f"✗ ERROR ({e})")
        results.append(
            {
                "parameter": param_name,
                "status_code": None,
                "ok": False,
                "message": f"Exception: {e}",
            }
        )

### Call NASA POWER API

In [1]:
import requests
import pandas as pd
import datetime
import time

In [5]:
params_df = pd.read_csv(r"F:\University\Year 3 Semester 1\CSS342 - Data Sci & Data En\Project_V2\Climate Change\Dataset\nasa_daily_parameters.csv")
valid_params = params_df["parameter"].tolist()

print(f"Number of parameters is {len(valid_params)}")

PARAMETERS_AFTER_2001 = [
    "CLRSKY_SFC_PAR_DIRH",
    "CLRSKY_SFC_PAR_DIFF", 
    "CLRSKY_SFC_LW_UP",
    "CLRSKY_SFC_LW_DWN",
    "CLRSKY_NKT",
    "CLRSKY_KT", 
    "CLRSKY_DAYS",
    "CLOUD_OD",
    "CLOUD_AMT_NIGHT",
    "CLOUD_AMT_DAY",
    "CLOUD_AMT",
    "CDD18_3"
]

Number of parameters is 129


In [6]:
LAT = 13.736717
LON = 100.523186

power_url = "https://power.larc.nasa.gov/api/temporal/daily/point"

# start date default
DEFAULT_START = "19810101"
AFTER_2001_START = "20010101"
end_date = datetime.datetime.today().strftime("%Y%m%d")

In [7]:
all_param_data = {}

def chunk_list(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [8]:
for batch_idx, batch_params in enumerate(chunk_list(valid_params, 20), start=1):

    # ตรวจว่า batch มี param ที่เริ่มหลังปี 2001 ไหม
    has_after_2001 = any(p in PARAMETERS_AFTER_2001 for p in batch_params)

    # เลือก start_date ตามประเภทของ batch
    if has_after_2001:
        start_date = AFTER_2001_START
    else:
        start_date = DEFAULT_START

    param_str = ",".join(batch_params)

    request_params = {
        "parameters": param_str,
        "community": "AG",
        "longitude": LON,
        "latitude": LAT,
        "start": start_date,
        "end": end_date,
        "format": "JSON"
    }

    print(f"\n=== Batch {batch_idx} | {len(batch_params)} parameters ===")
    print(f"Parameters: {param_str}")

    if has_after_2001:
        print(f"Batch {batch_idx}. There are parameters that start after 2001 → change start_date to {AFTER_2001_START}")
    else:
        print(f"Batch {batch_idx} use the full year period from {DEFAULT_START}")

    try:
        r = requests.get(power_url, params=request_params, timeout=60)
        print(f"Status Code: {r.status_code}")

        if r.status_code == 422:
            print("  ↳ 422 Unprocessable Entity:")
            print(r.text[:300])
            continue

        r.raise_for_status()

        data = r.json()
        records = data["properties"]["parameter"]

        # รวมข้อมูลเข้า dictionary ใหญ่
        for p_name, series_dict in records.items():
            all_param_data[p_name] = series_dict

        print(f"  ✓ Batch {batch_idx} OK, collected {len(records)} parameter(s)")

    except Exception as e:
        print(f"  ✗ Error in batch {batch_idx}: {e}")
        continue


=== Batch 1 | 20 parameters ===
Parameters: AIRMASS,ALLSKY_KT,ALLSKY_NKT,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_LW_UP,ALLSKY_SFC_PAR_DIFF,ALLSKY_SFC_PAR_DIRH,ALLSKY_SFC_PAR_TOT,ALLSKY_SFC_SW_DIFF,ALLSKY_SFC_SW_DIRH,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_SW_UP,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX,ALLSKY_SRF_ALB,AOD_55,AOD_55_ADJ,AOD_84
Batch 1 use the full year period from 19810101
Status Code: 200
  ✓ Batch 1 OK, collected 20 parameter(s)

=== Batch 2 | 20 parameters ===
Parameters: CDD18_3,CLOUD_AMT,CLOUD_AMT_DAY,CLOUD_AMT_NIGHT,CLOUD_OD,CLRSKY_DAYS,CLRSKY_KT,CLRSKY_NKT,CLRSKY_SFC_LW_DWN,CLRSKY_SFC_LW_UP,CLRSKY_SFC_PAR_DIFF,CLRSKY_SFC_PAR_DIRH,CLRSKY_SFC_PAR_TOT,CLRSKY_SFC_SW_DIFF,CLRSKY_SFC_SW_DIRH,CLRSKY_SFC_SW_DNI,CLRSKY_SFC_SW_DWN,CLRSKY_SFC_SW_UP,CLRSKY_SRF_ALB,DISPH
Batch 2. There are parameters that start after 2001 → change start_date to 20010101
Status Code: 200
  ✓ Batch 2 OK, collected 20 parameter(s)

=== Batch 3 | 20 parameters ===
Parameters: EVLAND,EVPTRNS,F

In [9]:
df_list = []

for param, series_dict in all_param_data.items():
    s = pd.Series(series_dict, name=param)
    s.index = pd.to_datetime(s.index, format="%Y%m%d", errors="coerce")
    df_list.append(s)

# Combine all Series into a single DataFrame (by date)
df = pd.concat(df_list, axis=1)

# sort by index
df = df.sort_index()

print("\n=== FINAL DATAFRAME SUMMARY ===")
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Date range: {df.index.min().date()} → {df.index.max().date()}")
print("\nColumns:")
print(list(df.columns))


=== FINAL DATAFRAME SUMMARY ===
Shape: 16,396 rows × 129 columns
Date range: 1981-01-01 → 2025-11-21

Columns:
['AIRMASS', 'ALLSKY_KT', 'ALLSKY_NKT', 'ALLSKY_SFC_LW_DWN', 'ALLSKY_SFC_LW_UP', 'ALLSKY_SFC_PAR_DIFF', 'ALLSKY_SFC_PAR_DIRH', 'ALLSKY_SFC_PAR_TOT', 'ALLSKY_SFC_SW_DIFF', 'ALLSKY_SFC_SW_DIRH', 'ALLSKY_SFC_SW_DNI', 'ALLSKY_SFC_SW_DWN', 'ALLSKY_SFC_SW_UP', 'ALLSKY_SFC_UVA', 'ALLSKY_SFC_UVB', 'ALLSKY_SFC_UV_INDEX', 'ALLSKY_SRF_ALB', 'AOD_55', 'AOD_55_ADJ', 'AOD_84', 'CDD18_3', 'CLOUD_AMT', 'CLOUD_AMT_DAY', 'CLOUD_AMT_NIGHT', 'CLOUD_OD', 'CLRSKY_DAYS', 'CLRSKY_KT', 'CLRSKY_NKT', 'CLRSKY_SFC_LW_DWN', 'CLRSKY_SFC_LW_UP', 'CLRSKY_SFC_PAR_DIFF', 'CLRSKY_SFC_PAR_DIRH', 'CLRSKY_SFC_PAR_TOT', 'CLRSKY_SFC_SW_DIFF', 'CLRSKY_SFC_SW_DIRH', 'CLRSKY_SFC_SW_DNI', 'CLRSKY_SFC_SW_DWN', 'CLRSKY_SFC_SW_UP', 'CLRSKY_SRF_ALB', 'DISPH', 'EVLAND', 'EVPTRNS', 'FROST_DAYS', 'FRSEAICE', 'FRSNO', 'GWETPROF', 'GWETROOT', 'GWETTOP', 'GWM_HEIGHT', 'GWM_HEIGHT_ANOMALY', 'HDD0', 'HDD10', 'HDD18_3', 'IMERG_PRECL

In [10]:
df.to_csv("nasa_power_data.csv", index_label="DATE")

In [83]:
df

,AIRMASS,ALLSKY_KT,ALLSKY_NKT,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_LW_UP,ALLSKY_SFC_PAR_DIFF,ALLSKY_SFC_PAR_DIRH,ALLSKY_SFC_PAR_TOT,ALLSKY_SFC_SW_DIFF,ALLSKY_SFC_SW_DIRH,...,WS10M_RANGE,WS2M,WS2M_MAX,WS2M_MIN,WS2M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,Z0M
1981-01-01,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,2.20,2.11,3.52,1.41,2.11,4.04,4.98,3.05,1.93,0.04
1981-01-02,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,3.08,2.23,4.01,1.32,2.69,4.25,5.85,2.66,3.19,0.04
1981-01-03,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,3.61,2.83,4.83,1.64,3.19,5.30,7.30,3.73,3.57,0.04
1981-01-04,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,4.26,2.26,4.38,0.82,3.56,4.32,6.61,1.66,4.95,0.04
1981-01-05,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,2.54,1.24,2.19,0.22,1.97,2.37,3.54,0.21,3.33,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-16,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,1.22,3.51,4.00,2.93,1.07,6.29,7.01,5.15,1.86,0.07
2025-11-17,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,1.47,3.17,3.72,2.57,1.15,5.69,6.46,4.66,1.80,0.07
2025-11-18,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00
2025-11-19,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00


## OPEN METEO

In [1]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

In [2]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [3]:
import datetime

end_date = datetime.datetime.today()

In [4]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"

# Get current date for end_date
end_date = datetime.datetime.today().strftime("%Y-%m-%d")

params = {
	"latitude": 13.736717,
	"longitude": 100.523186,
	"start_date": "1981-01-01",
	"end_date": "2025-11-17",
	"daily": ["weather_code", "temperature_2m_mean", "temperature_2m_max", "temperature_2m_min", "apparent_temperature_mean", "apparent_temperature_max", "apparent_temperature_min", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "precipitation_sum", "rain_sum", "snowfall_sum", "precipitation_hours", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration", "soil_temperature_7_to_28cm_mean", "soil_temperature_28_to_100cm_mean", "soil_temperature_0_to_7cm_mean", "soil_temperature_0_to_100cm_mean", "soil_moisture_7_to_28cm_mean", "soil_moisture_28_to_100cm_mean", "soil_moisture_0_to_7cm_mean", "soil_moisture_0_to_100cm_mean", "vapour_pressure_deficit_max", "wet_bulb_temperature_2m_min", "wet_bulb_temperature_2m_max", "wet_bulb_temperature_2m_mean", "pressure_msl_min", "pressure_msl_max", "snowfall_water_equivalent_sum", "pressure_msl_mean", "relative_humidity_2m_min", "relative_humidity_2m_max", "relative_humidity_2m_mean", "et0_fao_evapotranspiration_sum", "cloud_cover_mean", "cloud_cover_max", "cloud_cover_min", "dew_point_2m_mean", "dew_point_2m_max", "dew_point_2m_min", "surface_pressure_mean", "surface_pressure_max", "surface_pressure_min", "winddirection_10m_dominant", "wind_gusts_10m_mean", "wind_speed_10m_mean", "wind_gusts_10m_min", "wind_speed_10m_min"],
	"timezone": "Asia/Bangkok",
}

In [8]:
# สร้าง DataFrame
df = pd.DataFrame({"parameter": params["daily"]})

# Export เป็น CSV
df.to_csv("openmeteo_daily_parameters.csv", index=False)

In [5]:
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(2).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(3).ValuesAsNumpy()
daily_apparent_temperature_mean = daily.Variables(4).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(5).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(6).ValuesAsNumpy()
daily_sunrise = daily.Variables(7).ValuesInt64AsNumpy()
daily_sunset = daily.Variables(8).ValuesInt64AsNumpy()
daily_daylight_duration = daily.Variables(9).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(10).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(11).ValuesAsNumpy()
daily_rain_sum = daily.Variables(12).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(13).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(14).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(15).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(16).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(17).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(18).ValuesAsNumpy()
daily_et0_fao_evapotranspiration = daily.Variables(19).ValuesAsNumpy()
daily_soil_temperature_7_to_28cm_mean = daily.Variables(20).ValuesAsNumpy()
daily_soil_temperature_28_to_100cm_mean = daily.Variables(21).ValuesAsNumpy()
daily_soil_temperature_0_to_7cm_mean = daily.Variables(22).ValuesAsNumpy()
daily_soil_temperature_0_to_100cm_mean = daily.Variables(23).ValuesAsNumpy()
daily_soil_moisture_7_to_28cm_mean = daily.Variables(24).ValuesAsNumpy()
daily_soil_moisture_28_to_100cm_mean = daily.Variables(25).ValuesAsNumpy()
daily_soil_moisture_0_to_7cm_mean = daily.Variables(26).ValuesAsNumpy()
daily_soil_moisture_0_to_100cm_mean = daily.Variables(27).ValuesAsNumpy()
daily_vapour_pressure_deficit_max = daily.Variables(28).ValuesAsNumpy()
daily_wet_bulb_temperature_2m_min = daily.Variables(29).ValuesAsNumpy()
daily_wet_bulb_temperature_2m_max = daily.Variables(30).ValuesAsNumpy()
daily_wet_bulb_temperature_2m_mean = daily.Variables(31).ValuesAsNumpy()
daily_pressure_msl_min = daily.Variables(32).ValuesAsNumpy()
daily_pressure_msl_max = daily.Variables(33).ValuesAsNumpy()
daily_snowfall_water_equivalent_sum = daily.Variables(34).ValuesAsNumpy()
daily_pressure_msl_mean = daily.Variables(35).ValuesAsNumpy()
daily_relative_humidity_2m_min = daily.Variables(36).ValuesAsNumpy()
daily_relative_humidity_2m_max = daily.Variables(37).ValuesAsNumpy()
daily_relative_humidity_2m_mean = daily.Variables(38).ValuesAsNumpy()
daily_et0_fao_evapotranspiration_sum = daily.Variables(39).ValuesAsNumpy()
daily_cloud_cover_mean = daily.Variables(40).ValuesAsNumpy()
daily_cloud_cover_max = daily.Variables(41).ValuesAsNumpy()
daily_cloud_cover_min = daily.Variables(42).ValuesAsNumpy()
daily_dew_point_2m_mean = daily.Variables(43).ValuesAsNumpy()
daily_dew_point_2m_max = daily.Variables(44).ValuesAsNumpy()
daily_dew_point_2m_min = daily.Variables(45).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration
daily_data["soil_temperature_7_to_28cm_mean"] = daily_soil_temperature_7_to_28cm_mean
daily_data["soil_temperature_28_to_100cm_mean"] = daily_soil_temperature_28_to_100cm_mean
daily_data["soil_temperature_0_to_7cm_mean"] = daily_soil_temperature_0_to_7cm_mean
daily_data["soil_temperature_0_to_100cm_mean"] = daily_soil_temperature_0_to_100cm_mean
daily_data["soil_moisture_7_to_28cm_mean"] = daily_soil_moisture_7_to_28cm_mean
daily_data["soil_moisture_28_to_100cm_mean"] = daily_soil_moisture_28_to_100cm_mean
daily_data["soil_moisture_0_to_7cm_mean"] = daily_soil_moisture_0_to_7cm_mean
daily_data["soil_moisture_0_to_100cm_mean"] = daily_soil_moisture_0_to_100cm_mean
daily_data["vapour_pressure_deficit_max"] = daily_vapour_pressure_deficit_max
daily_data["wet_bulb_temperature_2m_min"] = daily_wet_bulb_temperature_2m_min
daily_data["wet_bulb_temperature_2m_max"] = daily_wet_bulb_temperature_2m_max
daily_data["wet_bulb_temperature_2m_mean"] = daily_wet_bulb_temperature_2m_mean
daily_data["pressure_msl_min"] = daily_pressure_msl_min
daily_data["pressure_msl_max"] = daily_pressure_msl_max
daily_data["snowfall_water_equivalent_sum"] = daily_snowfall_water_equivalent_sum
daily_data["pressure_msl_mean"] = daily_pressure_msl_mean
daily_data["relative_humidity_2m_min"] = daily_relative_humidity_2m_min
daily_data["relative_humidity_2m_max"] = daily_relative_humidity_2m_max
daily_data["relative_humidity_2m_mean"] = daily_relative_humidity_2m_mean
daily_data["et0_fao_evapotranspiration_sum"] = daily_et0_fao_evapotranspiration_sum
daily_data["cloud_cover_mean"] = daily_cloud_cover_mean
daily_data["cloud_cover_max"] = daily_cloud_cover_max
daily_data["cloud_cover_min"] = daily_cloud_cover_min
daily_data["dew_point_2m_mean"] = daily_dew_point_2m_mean
daily_data["dew_point_2m_max"] = daily_dew_point_2m_max
daily_data["dew_point_2m_min"] = daily_dew_point_2m_min

daily_dataframe = pd.DataFrame(data = daily_data)

Coordinates: 13.743409156799316°N 100.49586486816406°E
Elevation: 9.0 m asl
Timezone: b'Asia/Bangkok'b'GMT+7'
Timezone difference to GMT+0: 25200s


In [6]:
daily_dataframe.columns

Index(['date', 'weather_code', 'temperature_2m_mean', 'temperature_2m_max',
       'temperature_2m_min', 'apparent_temperature_mean',
       'apparent_temperature_max', 'apparent_temperature_min', 'sunrise',
       'sunset', 'daylight_duration', 'sunshine_duration', 'precipitation_sum',
       'rain_sum', 'snowfall_sum', 'precipitation_hours', 'wind_speed_10m_max',
       'wind_gusts_10m_max', 'wind_direction_10m_dominant',
       'shortwave_radiation_sum', 'et0_fao_evapotranspiration',
       'soil_temperature_7_to_28cm_mean', 'soil_temperature_28_to_100cm_mean',
       'soil_temperature_0_to_7cm_mean', 'soil_temperature_0_to_100cm_mean',
       'soil_moisture_7_to_28cm_mean', 'soil_moisture_28_to_100cm_mean',
       'soil_moisture_0_to_7cm_mean', 'soil_moisture_0_to_100cm_mean',
       'vapour_pressure_deficit_max', 'wet_bulb_temperature_2m_min',
       'wet_bulb_temperature_2m_max', 'wet_bulb_temperature_2m_mean',
       'pressure_msl_min', 'pressure_msl_max', 'snowfall_water_equiva

In [7]:
daily_dataframe.to_csv("openmeteo_climate_data.csv", index = False)

## Merge NASA POWER and OPEN-METEO

In [15]:
nasa_power_df = pd.read_csv("nasa_power_data.csv", parse_dates=["DATE"])
openmeteo_df = pd.read_csv("openmeteo_climate_data.csv", parse_dates=["date"])

In [16]:
nasa_power_df.columns

Index(['DATE', 'AIRMASS', 'ALLSKY_KT', 'ALLSKY_NKT', 'ALLSKY_SFC_LW_DWN',
       'ALLSKY_SFC_LW_UP', 'ALLSKY_SFC_PAR_DIFF', 'ALLSKY_SFC_PAR_DIRH',
       'ALLSKY_SFC_PAR_TOT', 'ALLSKY_SFC_SW_DIFF',
       ...
       'WS10M_RANGE', 'WS2M', 'WS2M_MAX', 'WS2M_MIN', 'WS2M_RANGE', 'WS50M',
       'WS50M_MAX', 'WS50M_MIN', 'WS50M_RANGE', 'Z0M'],
      dtype='object', length=130)

In [17]:
nasa_power_df.columns = nasa_power_df.columns.str.lower()
openmeteo_df.columns

Index(['date', 'weather_code', 'temperature_2m_mean', 'temperature_2m_max',
       'temperature_2m_min', 'apparent_temperature_mean',
       'apparent_temperature_max', 'apparent_temperature_min', 'sunrise',
       'sunset', 'daylight_duration', 'sunshine_duration', 'precipitation_sum',
       'rain_sum', 'snowfall_sum', 'precipitation_hours', 'wind_speed_10m_max',
       'wind_gusts_10m_max', 'wind_direction_10m_dominant',
       'shortwave_radiation_sum', 'et0_fao_evapotranspiration',
       'soil_temperature_7_to_28cm_mean', 'soil_temperature_28_to_100cm_mean',
       'soil_temperature_0_to_7cm_mean', 'soil_temperature_0_to_100cm_mean',
       'soil_moisture_7_to_28cm_mean', 'soil_moisture_28_to_100cm_mean',
       'soil_moisture_0_to_7cm_mean', 'soil_moisture_0_to_100cm_mean',
       'vapour_pressure_deficit_max', 'wet_bulb_temperature_2m_min',
       'wet_bulb_temperature_2m_max', 'wet_bulb_temperature_2m_mean',
       'pressure_msl_min', 'pressure_msl_max', 'snowfall_water_equiva

In [18]:
openmeteo_df.columns

Index(['date', 'weather_code', 'temperature_2m_mean', 'temperature_2m_max',
       'temperature_2m_min', 'apparent_temperature_mean',
       'apparent_temperature_max', 'apparent_temperature_min', 'sunrise',
       'sunset', 'daylight_duration', 'sunshine_duration', 'precipitation_sum',
       'rain_sum', 'snowfall_sum', 'precipitation_hours', 'wind_speed_10m_max',
       'wind_gusts_10m_max', 'wind_direction_10m_dominant',
       'shortwave_radiation_sum', 'et0_fao_evapotranspiration',
       'soil_temperature_7_to_28cm_mean', 'soil_temperature_28_to_100cm_mean',
       'soil_temperature_0_to_7cm_mean', 'soil_temperature_0_to_100cm_mean',
       'soil_moisture_7_to_28cm_mean', 'soil_moisture_28_to_100cm_mean',
       'soil_moisture_0_to_7cm_mean', 'soil_moisture_0_to_100cm_mean',
       'vapour_pressure_deficit_max', 'wet_bulb_temperature_2m_min',
       'wet_bulb_temperature_2m_max', 'wet_bulb_temperature_2m_mean',
       'pressure_msl_min', 'pressure_msl_max', 'snowfall_water_equiva

In [19]:
nasa_power_df.head(3)

,date,airmass,allsky_kt,allsky_nkt,allsky_sfc_lw_dwn,allsky_sfc_lw_up,allsky_sfc_par_diff,allsky_sfc_par_dirh,allsky_sfc_par_tot,allsky_sfc_sw_diff,...,ws10m_range,ws2m,ws2m_max,ws2m_min,ws2m_range,ws50m,ws50m_max,ws50m_min,ws50m_range,z0m
0,1981-01-01,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,2.20,2.11,3.52,1.41,2.11,4.04,4.98,3.05,1.93,0.04
1,1981-01-02,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,3.08,2.23,4.01,1.32,2.69,4.25,5.85,2.66,3.19,0.04
2,1981-01-03,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,3.61,2.83,4.83,1.64,3.19,5.30,7.30,3.73,3.57,0.04


In [20]:
openmeteo_df.head(3)

,date,weather_code,temperature_2m_mean,temperature_2m_max,temperature_2m_min,apparent_temperature_mean,apparent_temperature_max,apparent_temperature_min,sunrise,sunset,...,relative_humidity_2m_min,relative_humidity_2m_max,relative_humidity_2m_mean,et0_fao_evapotranspiration_sum,cloud_cover_mean,cloud_cover_max,cloud_cover_min,dew_point_2m_mean,dew_point_2m_max,dew_point_2m_min
0,1980-12-31 17:00:00+00:00,1.0,24.886750,30.230500,18.880499,26.777720,32.127903,20.847643,347154093,347194880,...,39.850525,89.866710,63.845350,4.164230,4.833334,30.0,0.0,16.878416,18.9305,15.080501
1,1981-01-01 17:00:00+00:00,3.0,25.557580,30.380499,20.130499,27.584173,32.866203,21.796860,347240515,347281314,...,41.177307,84.378006,62.214336,3.758709,45.583332,96.0,0.0,17.199250,19.1805,15.680500
2,1981-01-02 17:00:00+00:00,3.0,25.815918,29.980500,21.880499,27.540330,32.081028,22.888521,347326936,347367748,...,44.494736,79.089320,61.355670,4.105995,54.083332,100.0,7.0,17.501333,18.6805,16.280499


In [21]:
nasa_power_df['date'] = pd.to_datetime(nasa_power_df['date']).dt.normalize()
openmeteo_df['date'] = (
    pd.to_datetime(openmeteo_df['date'])
      .dt.tz_convert(None)   # เอา timezone ออก (UTC → naive)
      .dt.normalize()         # ตั้งเวลาเป็น 00:00:00
)

In [22]:
nasa_power_df.head(3)

,date,airmass,allsky_kt,allsky_nkt,allsky_sfc_lw_dwn,allsky_sfc_lw_up,allsky_sfc_par_diff,allsky_sfc_par_dirh,allsky_sfc_par_tot,allsky_sfc_sw_diff,...,ws10m_range,ws2m,ws2m_max,ws2m_min,ws2m_range,ws50m,ws50m_max,ws50m_min,ws50m_range,z0m
0,1981-01-01,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,2.20,2.11,3.52,1.41,2.11,4.04,4.98,3.05,1.93,0.04
1,1981-01-02,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,3.08,2.23,4.01,1.32,2.69,4.25,5.85,2.66,3.19,0.04
2,1981-01-03,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,3.61,2.83,4.83,1.64,3.19,5.30,7.30,3.73,3.57,0.04


In [23]:
openmeteo_df.head(3)

,date,weather_code,temperature_2m_mean,temperature_2m_max,temperature_2m_min,apparent_temperature_mean,apparent_temperature_max,apparent_temperature_min,sunrise,sunset,...,relative_humidity_2m_min,relative_humidity_2m_max,relative_humidity_2m_mean,et0_fao_evapotranspiration_sum,cloud_cover_mean,cloud_cover_max,cloud_cover_min,dew_point_2m_mean,dew_point_2m_max,dew_point_2m_min
0,1980-12-31,1.0,24.886750,30.230500,18.880499,26.777720,32.127903,20.847643,347154093,347194880,...,39.850525,89.866710,63.845350,4.164230,4.833334,30.0,0.0,16.878416,18.9305,15.080501
1,1981-01-01,3.0,25.557580,30.380499,20.130499,27.584173,32.866203,21.796860,347240515,347281314,...,41.177307,84.378006,62.214336,3.758709,45.583332,96.0,0.0,17.199250,19.1805,15.680500
2,1981-01-02,3.0,25.815918,29.980500,21.880499,27.540330,32.081028,22.888521,347326936,347367748,...,44.494736,79.089320,61.355670,4.105995,54.083332,100.0,7.0,17.501333,18.6805,16.280499


In [24]:
# Merge แบบ outer (ครอบคลุมทุกปีทุกข้อมูล)
merged_df = pd.merge(
    nasa_power_df,
    openmeteo_df,
    on="date",
    how="outer",        # ใช้ outer เพราะข้อมูลอาจคนละช่วงเวลา
    suffixes=("_nasa", "_openmeteo")
)

merged_df = merged_df.sort_values("date").reset_index(drop=True)

merged_df.head()

,date,airmass,allsky_kt,allsky_nkt,allsky_sfc_lw_dwn,allsky_sfc_lw_up,allsky_sfc_par_diff,allsky_sfc_par_dirh,allsky_sfc_par_tot,allsky_sfc_sw_diff,...,relative_humidity_2m_min,relative_humidity_2m_max,relative_humidity_2m_mean,et0_fao_evapotranspiration_sum,cloud_cover_mean,cloud_cover_max,cloud_cover_min,dew_point_2m_mean,dew_point_2m_max,dew_point_2m_min
0,1980-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.850525,89.866710,63.845350,4.164230,4.833334,30.0,0.0,16.878416,18.930500,15.080501
1,1981-01-01,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,41.177307,84.378006,62.214336,3.758709,45.583332,96.0,0.0,17.199250,19.180500,15.680500
2,1981-01-02,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,44.494736,79.089320,61.355670,4.105995,54.083332,100.0,7.0,17.501333,18.680500,16.280499
3,1981-01-03,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,42.976818,83.482620,63.269394,3.602937,40.791668,99.0,2.0,17.463833,18.630499,16.080500
4,1981-01-04,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,42.690216,85.901740,65.520560,3.770867,40.708332,97.0,0.0,18.263832,20.230500,16.230500


In [25]:
merged_df.tail()

,date,airmass,allsky_kt,allsky_nkt,allsky_sfc_lw_dwn,allsky_sfc_lw_up,allsky_sfc_par_diff,allsky_sfc_par_dirh,allsky_sfc_par_tot,allsky_sfc_sw_diff,...,relative_humidity_2m_min,relative_humidity_2m_max,relative_humidity_2m_mean,et0_fao_evapotranspiration_sum,cloud_cover_mean,cloud_cover_max,cloud_cover_min,dew_point_2m_mean,dew_point_2m_max,dew_point_2m_min
16391,2025-11-16,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,60.994537,90.246666,80.659386,3.037103,72.375,100.0,16.0,22.412252,23.8435,21.293499
16392,2025-11-17,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16393,2025-11-18,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16394,2025-11-19,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16395,2025-11-20,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16396 entries, 0 to 16395
Columns: 176 entries, date to dew_point_2m_min
dtypes: datetime64[ns](1), float64(175)
memory usage: 22.0 MB


In [27]:
merged_df = merged_df[
    merged_df['date'] != pd.to_datetime("1980-12-31")
]

start = "2025-11-17"
end = "2025-11-20"

mask = (merged_df['date'] < start) | (merged_df['date'] > end)
merged_df = merged_df[mask]


In [28]:
print(merged_df['date'].min())
print(merged_df['date'].max())

1981-01-01 00:00:00
2025-11-16 00:00:00


In [31]:
merged_df.to_csv("not_clean_climate_data.csv", index=False)